In [ ]:
"""
Created on May 25, 2022 1:29 PM
author: melpakkampradeep
"""

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import io
import matplotlib
from matplotlib import pyplot as plt
import math
from sklearn.metrics import mean_squared_error, accuracy_score, confusion_matrix, precision_score, recall_score, accuracy_score

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, HalvingGridSearchCV
from sklearn.svm import SVC, LinearSVC
from sklearn.preprocessing import StandardScaler

from google.colab import files

pd.set_option("max_columns", None) # show all cols
pd.set_option('max_colwidth', None) # show full width of showing cols
pd.set_option("expand_frame_repr", False) # print cols side by side as it's supposed to be

In [ ]:
uploaded = files.upload()
# Read dataset (.csv format)
datafull = pd.read_csv(io.BytesIO(uploaded['COVID19_dataset_v4.csv']), na_filter=False)

Saving COVID19_dataset_v4.csv to COVID19_dataset_v4.csv


In [ ]:
X_t = datafull.drop(columns=['Wave', 'Date'])
y_t = datafull[['Wave']]

In [ ]:
# Use this split (or another split) if SVM takes too long to train
X_train = X_t.iloc[15409:51362]
X_test = X_t.iloc[60801:83000]

y_train = y_t.iloc[15409:51362]
y_test = y_t.iloc[60801:83000]

In [ ]:
countrylist = X_train.loc[:, 'Country_code'].unique()

In [ ]:
# Normalize train set for SVM
d_train = pd.DataFrame()

for c in countrylist[0:countrylist.size]:
    data = X_train[X_train.Country_code == c]
    data = data.drop(columns=["Country_code"])
    scaler1 = StandardScaler()
    data_s = scaler1.fit_transform(data)
    data_s = pd.DataFrame(data_s, columns = data.columns)
    d_train = pd.concat([data_s, d_train])

In [ ]:
countrylist = X_test.loc[:, 'Country_code'].unique()

In [ ]:
# Normalize test set for SVM
d_test = pd.DataFrame()

for c in countrylist[0:countrylist.size]:
    data = X_test[X_test.Country_code == c]
    data = data.drop(columns=["Country_code"])
    scaler1 = StandardScaler()
    data_s = scaler1.fit_transform(data)
    data_s = pd.DataFrame(data_s, columns = data.columns)
    d_test = pd.concat([data_s, d_test])

In [ ]:
# Drop Country_code column
X_train = X_train.drop(columns=['Country_code'])
X_test = X_test.drop(columns=['Country_code'])

In [ ]:
# Alternate split if training takes too long
X_train = d_train[45265:60352]
y_train = datafull[['Wave']]
y_train = y_train[45265:60352]

X_test = d_test[3345:11616]
y_test = test[['Wave']]
y_test = y_test[3345:11616]

In [ ]:
# Edit this to generate the n-day feature vector
d_train = d_train[["T15","T16","T17","T18","T19","T20","T21"]]
d_test = d_test[["T15","T16","T17","T18","T19","T20","T21"]]

In [ ]:
# Train and fit the SVM
svm_cl = SVC()
svm_cl.fit(d_train, y_train)

# Predict
preds = svm_cl.predict(d_test)
predst = svm_cl.predict(d_train)

# Score
print("Test acc: ", accuracy_score(y_test, preds))
print("Test rec:", recall_score(y_test, preds))
print("Test pres:", precision_score(y_test, preds))
print()
print("Train acc: ", accuracy_score(y_train, predst))
print("Train rec:", recall_score(y_train, predst))
print("Train pres:", precision_score(y_train, predst))

In [ ]:
# Use this (or an alternate split) if hyperparameter tuning takes too long
X_train_hypt = X_train[45265:60352]
y_train_hypt = y_train[45265:60352]

In [ ]:
# Basic hyperparameter tuning search space
param_grid = {'C':[0.1,1,100,1000],
              'kernel':['rbf'],
              'gamma': [1, 0.1, 0.01, 0.001]}

In [ ]:
# Hyperprameter tuning using GridSearch
svm_cl = SVC()

# Init Grid Search
grid_cv = GridSearchCV(svm_cl, param_grid, n_jobs=-1, cv=3, scoring="recall", verbose=True)

# Fit
_ = grid_cv.fit(d_train, y_train)

In [ ]:
# SVM Classifier with some hyperparameter tuning
final_cl = SVC(
    **grid_cv.best_params_,
)

_ = final_cl.fit(d_train, y_train)

preds = final_cl.predict(d_test)
predst = final_cl.predict(d_train)

# Score
print("Test acc: ", accuracy_score(y_test, preds))
print("Test rec:", recall_score(y_test, preds))
print("Test pres:", precision_score(y_test, preds))
print()
print("Train acc: ", accuracy_score(y_train, predst))
print("Train rec:", recall_score(y_train, predst))
print("Train pres:", precision_score(y_train, predst))